In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "data/coco/annotations/instances_train2017_5labels.json",
                        "data/coco/train2017/")
register_coco_instances("my_dataset_val", {}, "data/coco/annotations/instances_val2017_5labels.json",
                        "data/coco/val2017/")
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 30000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.SOLVER.MAX_ITER = 300000

In [ ]:
trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
1+1

In [ ]:
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset


In [ ]:
import cv2

In [ ]:
im_name = "data/coco/val2017/000000001490.jpg"
im = cv2.imread(im_name)

In [ ]:
def conv(ins,num_classes):
    fields = ins.get_fields()
    boxes = fields['pred_boxes'].tensor.numpy()
    res = [[] for i in range(num_classes)]
    for i in range(0,len(fields['pred_classes'])):
        pred_class = fields['pred_classes'][i].item()
        probs = ins.get_fields()['scores'][i].item()
        box_cord = list(boxes[i])
        box_cord.append(probs)
        res[pred_class].append(box_cord)
    res2 = []
    for i in res:
        if len(i)==0:
            q = np.array(i,dtype=np.float32).reshape((0,num_classes))
        else:
            q = np.array(i,dtype=np.float32)
        res2.append(q)
    return res2
    return [np.array(i,dtype=np.float32) for i in res]
        

In [ ]:
import pickle

In [ ]:
dt = pickle.load(open("cocoval_results_fasterRCNN_5labels.pkl",'rb'))

In [ ]:
datas = pickle.load(open("cocoval_dataset_fasterRCNN_5labels.pkl",'rb'))

In [ ]:
results = []
for i in datas:
    im_name = "data/coco/val2017/"+i['seg_map'].replace(".png",'.jpg')
    im = cv2.imread(im_name)
    outputs = predictor(im)
    results.append(conv(outputs["instances"].to("cpu"),num_classes))
    if len(results)%100==0:
        print(len(results)/len(datas)*100,'% complete')

In [ ]:
pickle.dump(results,open("detectron_coco_5labels_inference_more_epochs.pkl",'wb'))